In [1]:
from surprise import NMF, Dataset, Reader, SVD, SVDpp
from surprise.model_selection import cross_validate, GridSearchCV
import pandas as pd
from collections import defaultdict

from UserDefinedNMF import UserDefinedNMF

In [2]:
def load_data(file_path):
    df = pd.read_csv(file_path, usecols=['userId', 'movieId', 'rating'])
    return df

In [3]:
def prepare_data_for_surprise(dataframe):
    reader = Reader(rating_scale=(1, 5))
    data = Dataset.load_from_df(dataframe[['userId', 'movieId', 'rating']], reader)
    return data

In [4]:
traindf = load_data('./datasets/training_data.csv')

data = prepare_data_for_surprise(traindf)

In [5]:
user_ids = traindf['userId'].unique()
user_id_map = {old: new for new, old in enumerate(user_ids)}
item_ids = traindf['movieId'].unique()
item_id_map = {old: new for new, old in enumerate(item_ids)}

traindf['userId'] = traindf['userId'].map(user_id_map)
traindf['movieId'] = traindf['movieId'].map(item_id_map)

In [6]:
model = UserDefinedNMF()

model.fit(data.build_full_trainset())

In [7]:
u, i = 1, 2
prediction = model.predict(u, i)
print(f"Predicted rating for user {u} and item {i} is {prediction}")

Predicted rating for user 1 and item 2 is 3.991949182498951


In [8]:
#param_grid = {
#    'n_factors': [15],
#    'n_epochs': [30],
#    #'init_mean': [0, 0.05],  #SVD feature 
#    #'init_std_dev': [0.05, 0.1, 0.15],  #SVD feature
#    #'lr_all': [0.005, 0.01],  #SVD feature 
#    #'reg_all': [0.02, 0.05],  #SVD feature 
#    'reg_pu' : [0.12],  #NMF feature
#    'reg_qi' : [0.12],  #NMF feature
#    'reg_bu' : [0.04],  #NMF feature
#    'reg_bi' : [0.02],  #NMF feature
#    'lr_bu' : [0.01],  #NMF feature
#    'lr_bi' : [0.01]  #NMF feature
#}
#
##gs = GridSearchCV(SVD, param_grid, measures = ['rmse', 'mae'], cv = 5)
#gs = GridSearchCV(NMF, param_grid, measures = ['rmse', 'mae'], cv = 5)
#
#gs.fit(data)
#
#print(f"Best RMSE score achieved: {gs.best_score['rmse']}")
#
#print(f"Best parameters: {gs.best_params['rmse']}")

In [9]:
testdf = load_data('./datasets/testing_data.csv')
testdf['userId'] = testdf['userId'].map(user_id_map, na_action='ignore')
testdf['movieId'] = testdf['movieId'].map(item_id_map, na_action='ignore')
trainset = data.build_full_trainset()
testset = [tuple(row) for row in testdf.itertuples(index=False)]

In [10]:
predictions = model.test(testset)

In [11]:
#algo = SVD(**gs.best_params['rmse'])
#algo = NMF(**gs.best_params['rmse'])
#
#algo.fit(trainset)
#
#predictions = algo.test(testset)

In [12]:
#def calculate_accuracy(predictions, threshold=0.5):
#    correct_predictions = sum(abs(pred.est - pred.r_ui) <= threshold for pred in predictions)
#    accuracy = correct_predictions / len(predictions)
#    return accuracy

In [13]:
def calculate_accuracy(predictions, threshold=0.5):
    correct_predictions = sum(abs(estimated_rating - true_rating) <= threshold for _, _, true_rating, estimated_rating in predictions)
    accuracy = correct_predictions / len(predictions)
    return accuracy

In [14]:
threshold = 0.5
test_accuracy = calculate_accuracy(predictions, threshold)

print(f"Accuracy (within ±{threshold} of actual rating): {test_accuracy:.2%}")

Accuracy (within ±0.5 of actual rating): 46.64%


In [15]:
def create_predictions_dataframe(predictions):
    predictions_data = [{'uid': pred[0], 'iid': pred[1], 'og_rating': pred[2], 'nmf_rating': pred[3]} for pred in predictions]
    return pd.DataFrame(predictions_data)

test_pred_df = create_predictions_dataframe(predictions)

In [16]:
df_movies = pd.read_csv('./datasets/Movies.csv')

In [17]:
def get_top_n(predictions, n=10):
    top_n = defaultdict(list)
    org_ratings = defaultdict(list)
    
    for row in predictions.itertuples(index=False):
        uid, iid, og_rating, nmf_rating = row.uid, row.iid, row.og_rating, row.nmf_rating
        top_n[uid].append((iid, nmf_rating))
        org_ratings[uid].append((iid, og_rating))
    
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)  
        top_n[uid] = user_ratings[:n]
    
    return top_n, org_ratings

In [18]:
def show_user_recommended_movies_in_df(predictions, user_id, n=10):
    top_n, org_ratings = get_top_n(predictions, n)
    
    if user_id in top_n:
        recommended_movies_ids = [movie_id for movie_id, _ in top_n[user_id]]
    else:
        return pd.DataFrame()
    
    recommended_movies_df = df_movies.loc[df_movies['movieId'].isin(recommended_movies_ids)]
    
    return recommended_movies_df

In [37]:
for i in range(1,611):
    print("\t\t\t\t*****{}*****\n".format(i))
    print(show_user_recommended_movies_in_df(test_pred_df, i, 5))
    print("\n")

				*****1*****

     movieId                                              title  \
143      171                                     Jeffrey (1995)   
168      199  Umbrellas of Cherbourg, The (Parapluies de Che...   
580      711                                     Flipper (1996)   

                              genres  
143              ['Comedy', 'Drama']  
168  ['Drama', 'Musical', 'Romance']  
580        ['Adventure', 'Children']  


				*****2*****

      movieId                   title                  genres
88        100        City Hall (1996)   ['Drama', 'Thriller']
619       784   Cable Guy, The (1996)  ['Comedy', 'Thriller']
1202     1600  She's So Lovely (1997)    ['Drama', 'Romance']
1716     2306         Holy Man (1998)              ['Comedy']


				*****3*****

      movieId                          title  \
50         55                 Georgia (1995)   
112       132                    Jade (1995)   
1577     2116  Lord of the Rings, The (1978)   
2156     2871     

     movieId                      title              genres
112      132                Jade (1995)        ['Thriller']
215      251         Hunted, The (1995)          ['Action']
251      290  Once Were Warriors (1994)  ['Crime', 'Drama']


				*****19*****

     movieId                                              title  \
280      321  Strawberry and Chocolate (Fresa y chocolate) (...   
282      324                              Sum of Us, The (1994)   
490      563                                    Germinal (1993)   
949     1250               Bridge on the River Kwai, The (1957)   

                            genres  
280                      ['Drama']  
282            ['Comedy', 'Drama']  
490           ['Drama', 'Romance']  
949  ['Adventure', 'Drama', 'War']  


				*****20*****

     movieId                                              title  \
168      199  Umbrellas of Cherbourg, The (Parapluies de Che...   
485      553                                   Tombstone (1993)  

      movieId                           title  \
389       448                 Fearless (1993)   
679       897  For Whom the Bell Tolls (1943)   
1208     1606       Kull the Conqueror (1997)   

                                        genres  
389                                  ['Drama']  
679   ['Adventure', 'Drama', 'Romance', 'War']  
1208                   ['Action', 'Adventure']  


				*****42*****

      movieId                                  title  \
44         48                      Pocahontas (1995)   
314       356                    Forrest Gump (1994)   
849      1119                          Drunks (1995)   
1164     1544  Lost World: Jurassic Park, The (1997)   

                                                 genres  
44    ['Animation', 'Children', 'Drama', 'Musical', ...  
314               ['Comedy', 'Drama', 'Romance', 'War']  
849                                           ['Drama']  
1164      ['Action', 'Adventure', 'Sci-Fi', 'Thriller']  


				*****43***

     movieId                                              title  \
54        61                              Eye for an Eye (1996)   
153      181     Mighty Morphin Power Rangers: The Movie (1995)   
624      791  Last Klezmer: Leopold Kozlowski, His Life and ...   

                     genres  
54    ['Drama', 'Thriller']  
153  ['Action', 'Children']  
624         ['Documentary']  


				*****61*****

      movieId                                           title  \
153       181  Mighty Morphin Power Rangers: The Movie (1995)   
1649     2202                                 Lifeboat (1944)   
1655     2208                       Lady Vanishes, The (1938)   

                                genres  
153             ['Action', 'Children']  
1649                  ['Drama', 'War']  
1655  ['Drama', 'Mystery', 'Thriller']  


				*****62*****

      movieId                            title  \
156       185                  Net, The (1995)   
489       562  Welcome to the Dollhouse (1995)

     movieId                       title                          genres
233      271        Losing Isaiah (1995)                       ['Drama']
389      448             Fearless (1993)                       ['Drama']
485      553            Tombstone (1993)  ['Action', 'Drama', 'Western']
715      934  Father of the Bride (1950)                      ['Comedy']


				*****82*****

      movieId                            title                 genres
227       263         Ladybird Ladybird (1994)              ['Drama']
499       579  Escort, The (Scorta, La) (1993)  ['Crime', 'Thriller']
2000     2661  It Came from Outer Space (1953)             ['Sci-Fi']


				*****83*****

      movieId                                              title  \
3           4                           Waiting to Exhale (1995)   
215       251                                 Hunted, The (1995)   
217       253  Interview with the Vampire: The Vampire Chroni...   
2000     2661                    It Came fr

...

      movieId                               title  \
11         12  Dracula: Dead and Loving It (1995)   
12         13                        Balto (1995)   
112       132                         Jade (1995)   
2215     2943                    Indochine (1992)   

                                      genres  
11                      ['Comedy', 'Horror']  
12    ['Adventure', 'Animation', 'Children']  
112                             ['Thriller']  
2215                    ['Drama', 'Romance']  


				*****509*****

     movieId                                              title  \
3          4                           Waiting to Exhale (1995)   
54        61                              Eye for an Eye (1996)   
61        69                                      Friday (1995)   
280      321  Strawberry and Chocolate (Fresa y chocolate) (...   

                             genres  
3    ['Comedy', 'Drama', 'Romance']  
54            ['Drama', 'Thriller']  
61                       ['C

     movieId                title                                  genres
12        13         Balto (1995)  ['Adventure', 'Animation', 'Children']
26        27  Now and Then (1995)                   ['Children', 'Drama']
389      448      Fearless (1993)                               ['Drama']


				*****528*****

      movieId                 title                genres
331       373  Red Rock West (1992)          ['Thriller']
674       889     1-900 (06) (1994)  ['Drama', 'Romance']
1307     1753     Half Baked (1998)            ['Comedy']


				*****529*****

     movieId                             title  \
19        20                Money Train (1995)   
307      349   Clear and Present Danger (1994)   
389      448                   Fearless (1993)   
407      469  House of the Spirits, The (1993)   

                                                genres  
19   ['Action', 'Comedy', 'Crime', 'Drama', 'Thrill...  
307           ['Action', 'Crime', 'Drama', 'Thriller']  
389     

      movieId                        title  \
124       151               Rob Roy (1995)   
1978     2625  Black Mask (Hak hap) (1996)   

                                                 genres  
124               ['Action', 'Drama', 'Romance', 'War']  
1978  ['Action', 'Adventure', 'Crime', 'Sci-Fi', 'Th...  


				*****552*****

     movieId                       title                            genres
46        50  Usual Suspects, The (1995)  ['Crime', 'Mystery', 'Thriller']
79        88          Black Sheep (1996)                        ['Comedy']
374      430        Calendar Girl (1993)               ['Comedy', 'Drama']
607      762           Striptease (1996)               ['Comedy', 'Crime']


				*****553*****

     movieId                    title                               genres
50        55           Georgia (1995)                            ['Drama']
84        95      Broken Arrow (1996)  ['Action', 'Adventure', 'Thriller']
393      452      Widows' Peak (1994)        

     movieId             title                                  genres
12        13      Balto (1995)  ['Adventure', 'Animation', 'Children']
373      429  Cabin Boy (1994)                              ['Comedy']


				*****574*****

      movieId                                    title  \
598       743                          Spy Hard (1996)   
932      1232                           Stalker (1979)   
936      1236                             Trust (1990)   
1433     1956                   Ordinary People (1980)   
1723     2315  Bride of Chucky (Child's Play 4) (1998)   

                                genres  
598                         ['Comedy']  
932     ['Drama', 'Mystery', 'Sci-Fi']  
936     ['Comedy', 'Drama', 'Romance']  
1433                         ['Drama']  
1723  ['Comedy', 'Horror', 'Thriller']  


				*****575*****

      movieId                                  title  \
118       145                        Bad Boys (1995)   
611       773                     Touk

    movieId                               title  \
3         4            Waiting to Exhale (1995)   
11       12  Dracula: Dead and Loving It (1995)   

                            genres  
3   ['Comedy', 'Drama', 'Romance']  
11            ['Comedy', 'Horror']  


				*****597*****

     movieId             title  \
485      553  Tombstone (1993)   
559      673  Space Jam (1996)   

                                                genres  
485                     ['Action', 'Drama', 'Western']  
559  ['Adventure', 'Animation', 'Children', 'Comedy...  


				*****598*****

     movieId           title                                 genres
124      151  Rob Roy (1995)  ['Action', 'Drama', 'Romance', 'War']


				*****599*****

     movieId                               title                 genres
11        12  Dracula: Dead and Loving It (1995)   ['Comedy', 'Horror']
26        27                 Now and Then (1995)  ['Children', 'Drama']
227      263            Ladybird Ladybird (199

In [42]:
show_user_recommended_movies_in_df(test_pred_df, 609, 5)

,movieId,title,genres
12,13,Balto (1995),"['Adventure', 'Animation', 'Children']"
49,54,"Big Green, The (1995)","['Children', 'Comedy']"
659,858,"Godfather, The (1972)","['Crime', 'Drama']"
2086,2772,Detroit Rock City (1999),['Comedy']
